# MRC(Machine Reading Comprehension)

## Description

Usually given a doc or some docs, and given a question Q, aim to output answer A

There are some types pf MRC, such as 
- "Cloze test"
- "Answer selection"
- "Fragment extraction"
- "Freely generation"

In the below task, I choose the "Fragment extraction" cause I used Encoder model

## Fragment extraction

### 1-Sample

### 2-Data format

{\
&nbsp;&nbsp;&nbsp;&nbsp;"context:"《战国无双三》是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴\
分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》\
&nbsp;&nbsp;&nbsp;&nbsp;"id:""DEV_0_QUERY_0"\
&nbsp;&nbsp;&nbsp;&nbsp;"question:"《战国无双三》是由哪两个公司合作开发的？\
&nbsp;&nbsp;&nbsp;&nbsp;"answers:"{\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"answer start:"[11, 11],\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"text:"["光荣和ω-force", "光荣和ω-force"]\
&nbsp;&nbsp;&nbsp;&nbsp;},\
}

## Assess metrics

### EM(Exact Match) & F1

the two different metrics shows the "exact" and "fuzzy" accuracy of the prediction, given sample below:\
&nbsp;&nbsp;&nbsp;&nbsp;prediction: 北京\
&nbsp;&nbsp;&nbsp;&nbsp;label: 北京天安门\
&nbsp;&nbsp;compute:\
&nbsp;&nbsp;&nbsp;&nbsp;$EM$ = 0;\
&nbsp;&nbsp;&nbsp;&nbsp;$P$ = 2/2 = 1,&nbsp;$R$ = 2/5, $f1$ = $2*P*R$/$P+R$ = 4/7

In [1]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer, Pipeline, DefaultDataCollator
from datasets import *
import numpy as np
import evaluate

In [2]:
data = DatasetDict.load_from_disk('./mrc_data/')
print(data)
print(data['train'][0])
print(data['train'].features)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})
{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月2

In [3]:
tokenizer = AutoTokenizer.from_pretrained("./macbert-base-chinese/")
tokenizer

BertTokenizerFast(name_or_path='./macbert-base-chinese/', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [4]:
sample = data['train'][0]
tokenized_sample = tokenizer(text=sample['question'],
                             text_pair=sample['context'], 
                             return_offsets_mapping=True, # as the word_ids work
                             max_length=512,
                             truncation='only_second',
                             padding=True)

offset_mapping = tokenized_sample.pop("offset_mapping")
print(tokenized_sample)

{'input_ids': [101, 5745, 2455, 7563, 3221, 784, 720, 3198, 952, 6158, 818, 711, 712, 3136, 4638, 8043, 102, 5745, 2455, 7563, 3364, 3322, 8020, 8024, 8021, 8024, 1760, 1399, 924, 4882, 185, 5735, 4449, 8020, 8021, 8024, 3221, 6632, 1298, 5384, 7716, 1921, 712, 3136, 3364, 3322, 511, 9155, 2399, 6158, 818, 711, 712, 3136, 8039, 8431, 2399, 6158, 3091, 1285, 711, 1921, 712, 3136, 3777, 1079, 2600, 3136, 1277, 2134, 2429, 5392, 4415, 8039, 8447, 2399, 6158, 3091, 1285, 711, 2600, 712, 3136, 8024, 1398, 2399, 2399, 2419, 6158, 3091, 1285, 711, 3364, 3322, 8039, 8170, 2399, 123, 3299, 4895, 686, 511, 5745, 2455, 7563, 754, 9915, 2399, 127, 3299, 8115, 3189, 1762, 6632, 1298, 2123, 2398, 4689, 1921, 712, 3136, 1355, 5683, 3136, 1277, 1139, 4495, 8039, 4997, 2399, 3198, 2970, 1358, 5679, 1962, 3136, 5509, 1400, 8024, 6158, 671, 855, 6632, 1298, 4868, 4266, 2372, 1168, 3777, 1079, 5326, 5330, 1071, 2110, 689, 511, 5745, 2455, 7563, 754, 9211, 2399, 1762, 3777, 1079, 1920, 934, 6887, 7368, 213

In [5]:
print(offset_mapping)

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [6]:
 print("token answer decode:", tokenizer.decode(tokenized_sample["input_ids"]))

token answer decode: [CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 天 主 的 爱 」 。 由 于 范 廷 颂 被 越 南 政 府 软 禁 差 不 多 30 年 ， 因 此 他 无 法 到 所 属 堂 区 进 行 牧 灵 工 作 而 专 注 研 读 等 工 作 。 范 廷 颂 除 了 面 对 战 争 、 贫 困 、 被 当 局 迫 害 天 主 教 会 等 问 题 外 ， 也 秘 密 恢 复 修 院 、 创

In [9]:
print(data['train'][0]['context'][534])

理


In [10]:
def preprocess_func(example):
    # input tokenize
    tokenized_input = tokenizer(text=example['question'],
                                text_pair=example['context'],
                                return_offsets_mapping=True,
                                max_length=512,
                                truncation='only_second',
                                padding=True)
    # label extract
    offset_mapping = tokenized_input.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for idx, offset in enumerate(offset_mapping):
        # get answer strat idx and end idx ---- untokenied
        answer = example['answers'][idx]
        answer_start_ut = answer["answer_start"][0]
        answer_end_ut = answer_start_ut + len(answer['text'][0])

        # get answer within context idx
        context_start_t = tokenized_input.sequence_ids(idx).index(1) # sequence_ids() func show the [CLS],[SEP] and text-pair as [None, 0, 0, None, 1, 1, 1, None]
        context_end_t = tokenized_input.sequence_ids(idx).index(None, context_start_t) - 1

        # get answer strat idx and end idx ---- tokenied
        if answer_start_ut > offset[context_end_t][1] or answer_end_ut < offset[context_start_t][0]:
            answer_start_t = 0
            answer_end_t = 0
        else:
            temp_id = context_start_t
            while offset[temp_id][0] < answer_start_ut and temp_id <= context_end_t:
                # for the answer_start_ut is the last token
                temp_id += 1
            answer_start_t = temp_id
            temp_id = context_end_t
            while offset[temp_id][1] > answer_end_ut and temp_id >=context_start_t:
                temp_id -= 1
            answer_end_t = temp_id
        start_positions.append(answer_start_t)
        end_positions.append(answer_end_t)

    tokenized_input['start_positions'] = start_positions
    tokenized_input['end_positions'] = end_positions

    return tokenized_input

In [11]:
data = data.map(preprocess_func, batched=True, remove_columns=data['train'].column_names)
data

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})